# Initialization and StyleGAN2 importing

In [ ]:
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
from PIL import Image
from tqdm.notebook import tqdm
import warnings
import torch
import json

In [ ]:

 !git clone https://github.com/ArwaTaha1/stylegan2-ada-pytorch.git
#!pip install ninja

In [ ]:
!pip uninstall -y peft
!pip install torch==1.11.0 torchvision==0.12.0

# !pip install numpy scipy scikit-image pillow tqdm tensorboard matplotlib
# !pip install pyspng
# !pip install click

# Reformat all images to jpeg and RGB and resize to 512x512

In [ ]:

def clean_dataset(input_folder, output_folder, target_size=(512, 512)):
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in tqdm(os.listdir(input_folder)):
        input_path = os.path.join(input_folder, filename)
        name, _ = os.path.splitext(filename)
        output_path = os.path.join(output_folder, f"{name}.jpg")  # Save as JPG
        
        try:
            img = Image.open(input_path)
            img = img.convert("RGB")  # Ensure RGB mode
            img = img.resize(target_size, Image.LANCZOS)
            img.save(output_path, format="JPEG", quality=95)
        except Exception as e:
            print(f"Failed to process {input_path}: {e}")

clean_dataset("/kaggle/input/autism/autism/train/autistic", "/kaggle/working/data/gan/images/autistic")
clean_dataset("/kaggle/input/autism/autism/train/non_autistic", "/kaggle/working/data/gan/images/non_autistic")


# Check if all images are up to standard

In [ ]:
IMAGE_PATH = '/kaggle/working/data/gan/images/autistic'
IMAGE_PATH1 = '/kaggle/working/data/gan/images/non_autistic'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]
files1 = [f for f in listdir(IMAGE_PATH1) if isfile(join(IMAGE_PATH1, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz
      
base_size = None      
for file in tqdm(files1):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz

In [ ]:
# !rm -R /kaggle/working/kaggle

# Generate dataset.json manually

In [ ]:
dataset_root = "/kaggle/working/data/gan/images"
label_map = {"autistic": 0, "non_autistic": 1}
labels = []

for class_name, label in label_map.items():
    class_dir = os.path.join(dataset_root, class_name)
    for fname in os.listdir(class_dir):
        if fname.lower().endswith((".png", ".jpg", ".jpeg")):
            rel_path = f"{class_name}/{fname}"
            labels.append([rel_path, label])


with open(os.path.join(dataset_root, "dataset.json"), "w") as f:
    json.dump({"labels": labels}, f)

# Convert the images

In [ ]:
!python /kaggle/working/stylegan2-ada-pytorch/dataset_tool.py --source /kaggle/working/data/gan/images --dest /kaggle/working/data/gan/dataset.zip

In [ ]:
# !rm -R /kaggle/working/data
# !rm -R /kaggle/working/images_cleaned
# !rm -R /kaggle/working/seed0100.png

# Initial Training

In [ ]:
EXPERIMENTS = "/kaggle/working/data/gan/experiments"
DATA = "/kaggle/working/data/gan/dataset.zip"
SNAP = 10

cmd = f"/usr/bin/python3 /kaggle/working/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --cond=1 --mirror=1"
!{cmd}

# Resume Training

In [ ]:
EXPERIMENTS = "/kaggle/working/data/gan/experiments"
DATA = "/kaggle/working/data/gan/dataset.zip"
SNAP = 10
RESUME = '/kaggle/input/third-gan-run/data/gan/experiments/00000-dataset-cond-auto1-resumecustom/network-snapshot-000120.pkl'
cmd = f"/usr/bin/python3 /kaggle/working/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA} --cond=1 "
!{cmd}

# Generate images

In [ ]:
!python /kaggle/working/stylegan2-ada-pytorch/generate.py \
  --outdir /kaggle/working/ \
  --trunc=1 \
  --seeds=100 \
  --network=/kaggle/input/third-gan-run/data/gan/experiments/00000-dataset-cond-auto1-resumecustom/network-snapshot-000120.pkl \
  --class=0